In [1]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim

torch.manual_seed(1)

In [2]:
x1_train = torch.FloatTensor([[73], [93], [89], [96], [73]])
x2_train = torch.FloatTensor([[80], [88], [91], [98], [66]])
x3_train = torch.FloatTensor([[75], [93], [90], [100], [70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [3]:
# 가중치 w와 편향 b 초기화
w1 = torch.zeros(1, requires_grad=True)
w2 = torch.zeros(1, requires_grad=True)
w3 = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

 학습 단계에서 최적의 매개변수를 찾아내는 것이 옵티마이저의 역할. 여기서 최적이란, 손실 함수가 최솟값이 될 때의 매개변수 값을 의미함.
 그러나 일반적인 문제의 손실 함수는 매우 복잡함. 이런 상황에서 기울기를 잘 이용하여 함수의 최솟값을 찾는 것이 바로 옵티마이저.

잔차를 줄이기 위해 파라미터를 업데이트 할 때에는 해당 파라미터가 얼마나 output에 영향을 주어 오차를 만들어 냈는지가 필요하다. 
역전파에서는 이를 위해서 순전파에서 거친 prediction과 target의 오차함수 f(Error)를 간단히 h=a*W+b 형태로 표현할 때 Weight에 대해 편미분하여 오차에 끼친 영향,
즉 변화량 a를 구하여 파라미터를 업데이트 한다. (여러개의 순차적인 nested function들을 통해 변화한 변화율들을 모두 구해 업데이트)

In [4]:
#optimizer 설정
optimizer = optim.SGD([w1, w2, w3, b], lr=1e-5)

epochs = 1000
for epoch in range(epochs + 1):
    
    # H(x) 가설 계산
    hx = x1_train * w1 + x2_train * w2 + x3_train * w3 + b
    
    # cost 계산 <= 순전파(Feedforward)를 통해 최종 출력값과 실제값의 오차를 확인
    cost = torch.mean((hx - y_train)**2)
    
    # cost로 H(x) 개선
    optimizer.zero_grad() # 기울기 0으로 초기화
    cost.backward()       # 역전파 단계(backward pass), 파라미터들의 에러에 대한 변화도를 계산하여 누적함
    optimizer.step()      # optimizer에게 cost function를 효율적으로 최소화 할 수 있게 파라미터 업데이트 요청 
    
    # 100번마다 로그 출력 
    if epoch % 100 == 0:
        print(f'Epoch {epoch:4d}/{epochs} w1: {w1.item():3f} w2: {w2.item():3f} w3: {w3.item():3f} b: {b.item():3f} Cost: {cost.item():.6f}')


Epoch    0/1000 w1: 0.294012 w2: 0.293600 w3: 0.297380 b: 0.003420 Cost: 29661.800781
Epoch  100/1000 w1: 0.673507 w2: 0.660968 w3: 0.676231 b: 0.007920 Cost: 1.563634
Epoch  200/1000 w1: 0.678946 w2: 0.654962 w3: 0.676784 b: 0.008070 Cost: 1.497607
Epoch  300/1000 w1: 0.684251 w2: 0.649122 w3: 0.677305 b: 0.008219 Cost: 1.435026
Epoch  400/1000 w1: 0.689426 w2: 0.643444 w3: 0.677795 b: 0.008367 Cost: 1.375730
Epoch  500/1000 w1: 0.694474 w2: 0.637923 w3: 0.678255 b: 0.008514 Cost: 1.319511
Epoch  600/1000 w1: 0.699398 w2: 0.632556 w3: 0.678686 b: 0.008659 Cost: 1.266222
Epoch  700/1000 w1: 0.704202 w2: 0.627338 w3: 0.679088 b: 0.008804 Cost: 1.215696
Epoch  800/1000 w1: 0.708889 w2: 0.622266 w3: 0.679464 b: 0.008948 Cost: 1.167818
Epoch  900/1000 w1: 0.713462 w2: 0.617334 w3: 0.679811 b: 0.009090 Cost: 1.122429
Epoch 1000/1000 w1: 0.717924 w2: 0.612539 w3: 0.680133 b: 0.009232 Cost: 1.079378


In [5]:
# 행렬 연산을 통한 구현
# 훈련 데이터 또한 행렬로 선언
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  80], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

In [6]:
print(x_train.shape)
print(y_train.shape)

torch.Size([5, 3])
torch.Size([5, 1])


In [8]:
# 가중치와 편향 선언
W = torch.zeros((3, 1), requires_grad=True) # 행렬곱이 가능하도록 
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1e-5)

In [9]:
nb_epochs = 20
for epoch in range(nb_epochs + 1):
    
    # 행렬곱을 통한 가설 선언
    # 편향 b는 브로드 캐스팅되어 각 샘플에 더해짐 
    hx = x_train.matmul(W) + b

    # cost 계산
    cost = torch.mean((hx - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print(f'Epoch {epoch:4d}/{epochs} hx: {hx.squeeze().detach()} Cost: {cost.item():.6f}')

Epoch    0/1000 hx: tensor([0., 0., 0., 0., 0.]) Cost: 29661.800781
Epoch    1/1000 hx: tensor([66.7178, 80.1701, 76.1025, 86.0194, 61.1565]) Cost: 9537.694336
Epoch    2/1000 hx: tensor([104.5421, 125.6208, 119.2478, 134.7861,  95.8280]) Cost: 3069.590820
Epoch    3/1000 hx: tensor([125.9858, 151.3882, 143.7087, 162.4333, 115.4844]) Cost: 990.670288
Epoch    4/1000 hx: tensor([138.1429, 165.9963, 157.5768, 178.1071, 126.6283]) Cost: 322.481873
Epoch    5/1000 hx: tensor([145.0350, 174.2780, 165.4395, 186.9928, 132.9461]) Cost: 107.717064
Epoch    6/1000 hx: tensor([148.9423, 178.9730, 169.8976, 192.0301, 136.5279]) Cost: 38.687496
Epoch    7/1000 hx: tensor([151.1574, 181.6346, 172.4254, 194.8856, 138.5585]) Cost: 16.499043
Epoch    8/1000 hx: tensor([152.4131, 183.1435, 173.8590, 196.5043, 139.7097]) Cost: 9.365656
Epoch    9/1000 hx: tensor([153.1250, 183.9988, 174.6723, 197.4217, 140.3625]) Cost: 7.071114
Epoch   10/1000 hx: tensor([153.5285, 184.4835, 175.1338, 197.9415, 140.7325]